In [1]:
import os 
import glob
import numpy as np
import scipy as sp
import pandas as pd
from pandas import Series, DataFrame

In [20]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torchvision
from torchvision import datasets, models, transforms
from torchsummary import summary
from sklearn.metrics import confusion_matrix, accuracy_score
import random
import os
import time
from sklearn.model_selection import train_test_split
from PIL import Image

In [10]:
trainlists = pd.read_csv('./train_labels.csv')

,path,year,target
0,train_1111.png,1979,0
1,train_5746.png,1979,0
2,train_6365.png,1979,0
3,train_0152.png,1980,1
4,train_0928.png,1980,1


In [12]:
trainlist = trainlists['path'].values.tolist()
target = trainlists['target'].values.tolist()

In [21]:
#パスリストの作成
datapath_list = []
for i,j  in enumerate (trainlist):
    datapath_list.append('./train_images/'+ j)
    
npdatapath_list=np.array(datapath_list)
trainlist, testlist = train_test_split(npdatapath_list, test_size = 0.8)

In [28]:
for i in range(3):
    for i,f in enumerate(trainlist):
        im = Image.open(f)
        im.save('./image/'+str(1979+i)+'/'+str(j)+'.png', quality=100)

FileNotFoundError: [Errno 2] No such file or directory: './image/1980/./train_5850.png.png'

In [ ]:
def save_checkpoint(path, epoch, model):
    save_path = os.path.join(path, "model_epoch_{}.pkl".format(epoch))
    torch.save(model.state_dict(), save_path)
    print("Checkpoint saved to {}".format(save_path))

def load_checkpoint(model_dir,epoch,model):
    load_path = os.path.join(model_dir,"model_epoch_{}.pkl".format(epoch))
    checkpoint = torch.load(load_path)
    model.load_state_dict(checkpoint)
    print("Checkpoint loaded to {}".format(load_path))

In [ ]:
np.random.seed(1) 
random.seed(1) 
torch.manual_seed(1) 
torch.cuda.manual_seed(1)

In [ ]:
batchsize = 32 
epochs = 5 
epoch_start =1

In [ ]:
use_gpu = torch.cuda.is_available()
if use_gpu:
     print("cuda is available!")
     cudnn.benchmark = True
     cudnn.deterministic = True

In [ ]:
checkout_dir = "./checkout" 
if os.path.exists(checkout_dir) is False:
     os.mkdir(checkout_dir)

In [ ]:
n_classes = 40
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
model = models.resnet18(pretrained=True).to(device) 
n_filters = model.fc.in_features 
model.fc = nn.Linear(n_filters, n_classes)
if use_gpu:
    model.cuda()

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9, weight_decay=5e-4)
use_scheduler = False
if use_scheduler:
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
def train(model, train_loader,epoch):
    model.train()
    print('\nEpoch: %d' % epoch)
    train_loss = 0
    correct = 0
    total = 0
    if use_scheduler:
        scheduler.step()
    for batch_idx, (image, label) in enumerate(train_loader):
        image, label = image.to(device), label.to(device)
        optimizer.zero_grad()
        outputs = model(image)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += label.size(0)
        correct += predicted.eq(label).sum().item()
    print('Loss:{} | Acc:{} ({}/{})'.format((train_loss/(batch_idx+1)), 100.*correct/total, correct, total))

In [ ]:
def test(model, test_loader,epoch):
    model.eval()
    running_loss = 0
    test_loss = 0
    correct = 0
    total = 0
    for batch_idx,(image, label) in enumerate(test_loader):
        image, label = image.to(device), label.to(device)
        outputs = model(image)
        loss = criterion(outputs, label)
        test_loss += loss.item()
        _, predicted = outputs.max(1)
        total += label.size(0)
        correct += predicted.eq(label).sum().item()
    print('Loss:{} | Acc:{} ({}/{})'.format((test_loss/(batch_idx+1)), 100.*correct/total, correct, total))

In [ ]:
def evaluation(model_dir,epoch,model,test_loader):
    load_checkpoint(model_dir,epoch,model)
    model.eval()
    y_test = []
    y_pred = []
    for batch_idx,(image, label) in enumerate(test_loader):
        image, label = image.to(device), label.to(device)
        outputs = model(image)
        _, predictions = outputs.max(1)
        y_test.append(label.data.cpu().numpy())
        y_pred.append(predictions.data.cpu().numpy())
    y_test = np.concatenate(y_test)
    y_pred = np.concatenate(y_pred)
    print(accuracy_score(y_test, y_pred))

In [ ]:
class ImageTransform():
    def __init__(self,resize,mean,std):
        self.data_transform={
            'train':transforms.Compose([
            transforms.Resize([224, 224]),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ]),
            'test': transforms.Compose([
            transforms.Resize([224, 224]),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])}
            
    def __call__(self, img, phase ='train'):
            return self.data_transform[phase](img)

In [7]:



class yeardetectionDataset(data.Dataset):
    def __init__(self, datapath_list, transform=transform, phase = 'train'):
        self.file_list = file_list
        self.trainsform = transform
        self.phase = phase
        
    def __len__(self):
        return(len)
    
    def __getitem__(self, index):
        
        
        

SyntaxError: unexpected EOF while parsing (<ipython-input-7-79f5d05d4df2>, line 22)

In [ ]:
if __name__ == '__main__':
    summary(model, (3, 224, 224))
    data_transform = transforms.Compose([
       transforms.Resize([224, 224]),
       transforms.RandomHorizontalFlip(),
       transforms.ToTensor(),
       transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
     ])
    train_Dataset = datasets.ImageFolder(root='./fruits-360/Training',transform=data_transform)
    train_loader = torch.utils.data.DataLoader(train_Dataset,batch_size=batchsize, shuffle=True)
    test_Dataset = datasets.ImageFolder(root='./fruits-360/Test',transform=data_transform)
    test_loader = torch.utils.data.DataLoader(test_Dataset,batch_size=batchsize, shuffle=False)
    print("train images: {}".format(len(train_Dataset)))
    print("test images: {}".format(len(test_Dataset)))
    print("epoch: {}".format(epochs))
    print("batch size: {}".format(batchsize))

    for epoch in range(epoch_start, epochs+1):
        train(model, train_loader,epoch)
        test(model, test_loader,epoch)
        save_checkpoint(checkout_dir, epoch, model)
    evaluation(checkout_dir,epochs,model,test_loader)